## Decision Tree

In [1]:
import pandas as pd

c1 = ['low','high','low','low','low','high']
c2 = ['hot','hot','hot','cold','cold','cold']
c3 = [1,0,0,1,0,0]
df = pd.DataFrame({'wind':c1,'weather':c2,'ride':c3})
df = df[['weather', 'wind', 'ride']]

print df

  weather  wind  ride
0     hot   low     1
1     hot  high     0
2     hot   low     0
3    cold   low     1
4    cold   low     0
5    cold  high     0


In [2]:
# entropy function

import math

def entropy(data):
    
    # need to first find probabilities of different classes
    classes = {}
    targets = [i[-1] for i in data]
    
    for i in targets:
        if i not in classes:
            classes[i] = 1
        else:
            classes[i] += 1   
    
    # then use entropy definition
    probs = [float(i)/len(data) for i in classes.values()]
    entropy = -sum([probs[i]*math.log(probs[i],2) for i in range(len(probs))])

    return entropy

In [3]:
# choose what to split on

def what_to_split_on(data):
    
    num_features = len(data[0]) - 1
    baseEntropy = entropy(data)
    bestInfoGain = 0.0
    bestFeature = -1
    
    for i in range(num_features):
        
        row_vals = [x[i] for x in data]
        unique_vals = set(row_vals)
        newEntropy = 0.0
        for val in unique_vals:
            subset = split(data, i, val)
            
            prob = len(subset) / float(len(data))
            
            newEntropy += prob*entropy(subset)
            
        infoGain = baseEntropy - newEntropy
        if infoGain > bestInfoGain:
            bestInfoGain = infoGain
            bestFeature = i
    return bestFeature

In [4]:
# how to actually split the tree

def split(data, col_num, value):
    split_set = []
    for record in data:
        if record[col_num] == value:
            
            # get everything except that column
            reduced = record[:col_num]
            reduced.extend(record[col_num+1:])
            
            split_set.append(reduced)
    return split_set

In [9]:
[x[-1] for x in df.values]

[1, 0, 0, 1, 0, 0]

In [373]:
# it doesn't have to be recursive!

def buildTree(data, labels, rounds):
    
    # go until stopping criterion is reached
    while rounds > 0:
        
        rounds -= 1
        targets = [x[-1] for x in df.values]

        # base cases: if all targets have the same value, we are done
        if targets.count(targets[0]) == len(targets):
            print "all items in split group have same value"
            return targets[0]

        # now to main event, choose best feature
        bestFeature = what_to_split_on(data)
        bestFeatureLabel = labels[bestFeature]
        print "chose feature ", bestFeature

        # start to create tree with best feature at root
        tree = {bestFeatureLabel:{}}
        print "tree: ", tree

        del(labels[bestFeature])

        featValues = [x[bestFeature] for x in data]
        uniqueVals = set(featValues)
        for value in uniqueVals:
            tree[bestFeatureLabel][value] = buildTree(split(data, bestFeature, value),labels)
        return tree

In [369]:


def buildTree(data, labels):
    
    targets = [x[-1] for x in data]
    
    # base cases
    
    # if all targets have the same value, or if we are in a class of 1, we are done
    if targets.count(targets[0]) == len(targets):
        print "all items in split group have same value"
        return targets[0]
    if len(targets) == 1:
        print "one item in split group"
        return targets[0]
       
    # now to main event, choose best feature
    bestFeature = what_to_split_on(data)
    bestFeatureLabel = labels[bestFeature]
    
    print "chose feature ", bestFeature
    
    # start to create tree with best feature at root
    tree = {bestFeatureLabel:{}}
    
    print "tree: ", tree
    
    del(labels[bestFeature])
    
    featValues = [x[bestFeature] for x in data]
    uniqueVals = set(featValues)
    for value in uniqueVals:
        tree[bestFeatureLabel][value] = buildTree(split(data, bestFeature, value),labels)
    return tree

In [385]:
labels = ['weather','wind']
data = [list(df.values[i]) for i in range(len(df))]

buildTree(data, labels)

chose feature  1
tree:  {'wind': {}}
all items in split group have same value
chose feature  -1
tree:  {'weather': {}}
all items in split group have same value
all items in split group have same value


{'wind': {'high': 0, 'low': {'weather': {0: 0, 1: 1}}}}

In [ ]:
# not working:

def most(classlist):
    classcount={}
    for vote in classlist:
        if vote not in classcount.keys():
            classcount[vote]=0
        classcount[vote] += 1
    sortedClassCount=sorted(classcount.iteritems(),key=operator.itemgetter(1),reverse=True)
    return sortedClassCount[0][0]